In [ ]:
from pasteur.kedro.ipython import *
register_kedro()

In [ ]:
# sensitive
%pipe mimic_tab_admissions.ingest

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pasteur.metadata import Metadata
from pasteur.transform import TableTransformer, Attributes, get_dtype

view = "mimic_tab_admissions"
trn: TableTransformer = catalog.load(f"{view}.trn.table")
table: pd.DataFrame = catalog.load(f"{view}.wrk.idx_table")

In [ ]:
# sensitive
table.head()

In [ ]:
attrs = trn["idx"].get_attributes()

In [ ]:
from pasteur.synth.math import expand_table, calc_marginal, calc_marginal_1way, AttrSelector, AttrSelectors

In [ ]:
cols, cols_noncommon, domains = expand_table(attrs, table)

In [ ]:
x = [
    AttrSelector(0, {"dod_week": 0}),
    AttrSelector(0, {"dod_week": 0}),
    AttrSelector(0, {"dod_week": 0}),
    AttrSelector(0, {"dod_week": 0}),
    AttrSelector(0, {"dod_week": 0}),
    AttrSelector(0, {"dod_week": 0}),
]
%timeit calc_marginal_1way(cols, cols_noncommon, domains, x)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2294434/1456224199.py:9 in <cell line: 9>                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2294434/1456224199.py'                      │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:241 │
│ 8 in run_line_magic                                                                              │
│                                                                                                  │
│   2415 │   │   │   if getattr(fn, "needs_local_scope", False):                                   │
│   2416 │   │   │   │   kwargs['local_ns'] = self.get_local_scope(stack_depth)                    │
│   2417 │   │   │   with self.builtin_trap:                                                       │
│ ❱ 2418 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2419 │   │   │   return result                                                                 │
│   2420 │                                                                                         │
│   2421 │   def get_local_scope(self, stack_depth):                                               │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/decorator.py:232 in fun              │
│                                                                                                  │
│   229 │   │   def fun(*args, **kw):                                                              │
│   230 │   │   │   if not kwsyntax:                                                               │
│   231 │   │   │   │   args, kw = fix(args, kw, sig)                                              │
│ ❱ 232 │   │   │   return caller(func, *(extras + args), **kw)                                    │
│   233 │   fun.__name__ = func.__name__                                                           │
│   234 │   fun.__doc__ = func.__doc__                                                             │
│   235 │   fun.__wrapped__ = func                                                                 │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/magic.py:187 in         │
│ <lambda>                                                                                         │
│                                                                                                  │
│   184 │   # This is a closure to capture the magic_kind.  We could also use a class,             │
│   185 │   # but it's overkill for just that one bit of state.                                    │
│   186 │   def magic_deco(arg):                                                                   │
│ ❱ 187 │   │   call = lambda f, *a, **k: f(*a, **k)                                               │
│   188 │   │                                                                                      │
│   189 │   │   if callable(arg):                                                                  │
│   190 │   │   │   # "Naked" decorator call (just @foo, no args)                                  │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/magics/execution.py:118 │
│ 0 in timeit                                                                                      │
│                                                            

In [ ]:
%lprun -f calc_marginal_1way calc_marginal_1way(cols, cols_noncommon, domains, x)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2294434/3525935125.py:1 in <cell line: 1>                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2294434/3525935125.py'                      │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:241 │
│ 8 in run_line_magic                                                                              │
│                                                                                                  │
│   2415 │   │   │   if getattr(fn, "needs_local_scope", False):                                   │
│   2416 │   │   │   │   kwargs['local_ns'] = self.get_local_scope(stack_depth)                    │
│   2417 │   │   │   with self.builtin_trap:                                                       │
│ ❱ 2418 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2419 │   │   │   return result                                                                 │
│   2420 │                                                                                         │
│   2421 │   def get_local_scope(self, stack_depth):                                               │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/decorator.py:232 in fun              │
│                                                                                                  │
│   229 │   │   def fun(*args, **kw):                                                              │
│   230 │   │   │   if not kwsyntax:                                                               │
│   231 │   │   │   │   args, kw = fix(args, kw, sig)                                              │
│ ❱ 232 │   │   │   return caller(func, *(extras + args), **kw)                                    │
│   233 │   fun.__name__ = func.__name__                                                           │
│   234 │   fun.__doc__ = func.__doc__                                                             │
│   235 │   fun.__wrapped__ = func                                                                 │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/magic.py:187 in         │
│ <lambda>                                                                                         │
│                                                                                                  │
│   184 │   # This is a closure to capture the magic_kind.  We could also use a class,             │
│   185 │   # but it's overkill for just that one bit of state.                                    │
│   186 │   def magic_deco(arg):                                                                   │
│ ❱ 187 │   │   call = lambda f, *a, **k: f(*a, **k)                                               │
│   188 │   │                                                                                      │
│   189 │   │   if callable(arg):                                                                  │
│   190 │   │   │   # "Naked" decorator call (just @foo, no args)                                  │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/line_profiler/ipython_extension.py:1 │
│ 04 in lprun                                                                                      │
│                                                            

In [ ]:
table.columns


Index(['dod_year', 'dod_week', 'dod_day', 'admittime_year', 'admittime_week',
       'admittime_day', 'admittime_time', 'dischtime_year', 'dischtime_week',
       'dischtime_day', 'dischtime_time', 'deathtime_year', 'deathtime_week',
       'deathtime_day', 'deathtime_time', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'marital_status', 'ethnicity', 'hospital_expire_flag', 'gender'],
      dtype='object')


In [ ]:
import numexpr as ne

def calc_marginal_1way2(
    cols: dict[str, list[np.ndarray]],
    cols_noncommon: dict[str, list[np.ndarray]],
    domains: dict[str, list[int]],
    x: AttrSelectors,
    zero_fill: float | None = None,
):
    """Calculates the 1 way marginal of the subsections of attributes x"""

    # Find integer dtype based on domain
    dom = 1
    for attr in x:
        for i, (n, h) in enumerate(attr.cols.items()):
            dom *= domains[n][h] - (attr.common if i > 0 else 0)
    dtype = get_dtype(dom)

    n = len(next(iter(cols.values()))[0])
    _sum_nd = np.zeros((n,), dtype=np.int32)

    mul = 1
    for attr in reversed(x):
        for i, (n, h) in enumerate(attr.cols.items()):
            common = attr.common
            col = cols[n][h]
            col_nc = cols_noncommon[n][h]
            if i == 0 or common == 0:
                ne.evaluate("_sum_nd + col*mul", out=_sum_nd)
            else:
                ne.evaluate("_sum_nd + col_nc*mul", out=_sum_nd)

            mul *= domains[n][h] - (common if i > 0 else 0)

    counts = np.bincount(_sum_nd, minlength=dom)
    margin = counts.astype("float32")
    margin /= margin.sum()
    if zero_fill is not None:
        # Mutual info turns into NaN without this
        margin += zero_fill

    return margin.reshape(-1)

In [ ]:
# %lprun -f calc_marginal_1way calc_marginal_1way(cols, cols_noncommon, domains, x)
%timeit calc_marginal_1way2(cols, cols_noncommon, domains, x)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2294434/2036685144.py:2 in <cell line: 2>                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2294434/2036685144.py'                      │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:241 │
│ 8 in run_line_magic                                                                              │
│                                                                                                  │
│   2415 │   │   │   if getattr(fn, "needs_local_scope", False):                                   │
│   2416 │   │   │   │   kwargs['local_ns'] = self.get_local_scope(stack_depth)                    │
│   2417 │   │   │   with self.builtin_trap:                                                       │
│ ❱ 2418 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2419 │   │   │   return result                                                                 │
│   2420 │                                                                                         │
│   2421 │   def get_local_scope(self, stack_depth):                                               │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/decorator.py:232 in fun              │
│                                                                                                  │
│   229 │   │   def fun(*args, **kw):                                                              │
│   230 │   │   │   if not kwsyntax:                                                               │
│   231 │   │   │   │   args, kw = fix(args, kw, sig)                                              │
│ ❱ 232 │   │   │   return caller(func, *(extras + args), **kw)                                    │
│   233 │   fun.__name__ = func.__name__                                                           │
│   234 │   fun.__doc__ = func.__doc__                                                             │
│   235 │   fun.__wrapped__ = func                                                                 │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/magic.py:187 in         │
│ <lambda>                                                                                         │
│                                                                                                  │
│   184 │   # This is a closure to capture the magic_kind.  We could also use a class,             │
│   185 │   # but it's overkill for just that one bit of state.                                    │
│   186 │   def magic_deco(arg):                                                                   │
│ ❱ 187 │   │   call = lambda f, *a, **k: f(*a, **k)                                               │
│   188 │   │                                                                                      │
│   189 │   │   if callable(arg):                                                                  │
│   190 │   │   │   # "Naked" decorator call (just @foo, no args)                                  │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/magics/execution.py:118 │
│ 0 in timeit                                                                                      │
│                                                            